## Get Candlestick Data 

In [1]:
import requests
import calendar
import dateutil.parser as parser
from dateutil.relativedelta import relativedelta
from datetime import datetime, timezone
import yaml
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
now = datetime.now()

In [3]:
def convert_date(utc_time): 
    parsed_date = parser.parse(utc_time)
    var_date=parsed_date.date()
    var_time=parsed_date.time()
    var_f_time=var_time.hour
    var_julian_date=parsed_date.timetuple().tm_yday
    var_weekday=parsed_date.weekday()
    var_weekday_name=calendar.day_name[parsed_date.weekday()]
    return var_date, var_time, var_f_time, var_julian_date, var_weekday, var_weekday_name

In [4]:
with open ('back_test_pipeline_settings.yaml') as ymlfile:
    cfg = yaml.safe_load(ymlfile)
    instrument = cfg['currency']['instrument']

In [5]:
print('Instrument:', instrument)

Instrument: EUR_USD


### The Configs for Run:

In [6]:
with open ('config.yml') as ymlfile:
    cfg = yaml.safe_load(ymlfile)
    oanda_api_key = cfg['creds']['oanda_api']
    account_number = cfg['creds']['account_number'] 

### Select the Currency Pair

In [7]:
Load_10K_Records=True
Load_20K_Records=False

#  'EUR_USD','USD_CAD','EUR_GBP','EUR_AUD','EUR_CHF',
#  'GBP_USD','GBP_CHF','GBP_NZD','GBP_AUD','GBP_CAD',
#  'AUD_CAD','AUD_CHF','AUD_NZD','NZD_USD','EUR_CAD',
#  'USD_CHF','CAD_CHF','NZD_CAD','AUD_USD','EUR_NZD',
#  'NZD_CHF',

currency_pairs = [instrument]


timeframe = "H4"
#D #H1 #H4 M30
# https://developer.oanda.com/rest-live-v20/instrument-df/#CandlestickGranularity
price_char = "M"
#M(midpoint candles) #B(bid candles) #A(ask candles) #BA
price_com = "mid"
#mid #bid #ask

# def of OANDA request variable
provider_api_url = 'https://api-fxpractice.oanda.com/v3/accounts/{}/orders'.format(account_number)
request_headers = {
    "Authorization": oanda_api_key,
    "Accept-Datetime-Format": "RFC3339",
    "Connection": "Keep-Alive",
    "Content-Type": "application/json;charset=UTF-8"
}


In [8]:
provider_authorization = 'Bearer {0}'.format(oanda_api_key)

headers = {
    'Content-Type': 'application/json',
    'Authorization': provider_authorization,
}

# Get Candlesticks Data

In [9]:
params_count = (
    ('price', price_char),
    ('count', '5000'),
    ('granularity', timeframe),
)


In [10]:
for pair in currency_pairs:
    first_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair), 
                            headers=headers,
                            params=params_count).json()

In [11]:
if Load_10K_Records:
    
    #### Load 10K data
        
    datetime_object = parser.parse(first_response['candles'][0]['time'])
    date= datetime_object - relativedelta(years=3)  
    from_date = date.replace(tzinfo=timezone.utc).timestamp()
    params_date = (
        ('count', '5000'),
        ('price', price_char),
        ('from', from_date),
        ('granularity', timeframe),)

    second_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair),
                                   headers=headers,
                                   params=params_date).json()
    datetime_object_15K = parser.parse(second_response['candles'][0]['time'])
    first_response= first_response['candles']  
    second_response= second_response['candles']
    second_response.extend(first_response)
    
    if Load_20K_Records:

        #### Load 15K data

        date= datetime_object_15K - relativedelta(years=3)  
        from_date = date.replace(tzinfo=timezone.utc).timestamp()
        params_date = (
            ('count', '5000'),
            ('price', price_char),
            ('from', from_date),
            ('granularity', timeframe),)

        third_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair),
                                       headers=headers,
                                       params=params_date).json()
        datetime_object_20K = parser.parse(third_response['candles'][0]['time'])
        third_response= third_response['candles']
        third_response.extend(second_response)

        #### Load 20K data

        date= datetime_object_20K - relativedelta(years=3)  
        from_date = date.replace(tzinfo=timezone.utc).timestamp()
        params_date = (
            ('count', '5000'),
            ('price', price_char),
            ('from', from_date),
            ('granularity', timeframe),)

        fourth_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair),
                                       headers=headers,
                                       params=params_date).json()
        datetime_object_20K = parser.parse(fourth_response['candles'][0]['time'])
        fourth_response= fourth_response['candles']
        fourth_response.extend(third_response)

        response=fourth_response
    else:
       response=second_response 
else:
    response=first_response['candles']

In [12]:
filename = "{}_{}.csv".format(pair, timeframe)
output = []
all_candlesticks = response

for i in range (len(all_candlesticks)):
    result= (convert_date(response[i]['time']))
    output.append([(result[0]),(result[1]),(result[2]),(result[3]),(result[4]),(result[5]),
                    response[i]['time'],
                    response[i]['volume'], 
                    response[i][price_com]['o'],
                    response[i][price_com]['h'],
                    response[i][price_com]['l'],
                    response[i][price_com]['c']])
    
output = pd.DataFrame(output)
output.columns = ['Date','Time','f_time','julian_date','Weekday','Weekday_Name','UTC_Time', 'Volume', 'Open', 'High', 'Low', 'Close']
data = output.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [13]:
data = data.drop_duplicates()
data = data.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [14]:
data.shape

(9659, 12)

In [15]:
data.head(5)

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close
0,2015-11-29,22:00:00,22,333,6,Sunday,2015-11-29T22:00:00.000000000Z,2657,1.05902,1.05945,1.05706,1.05794
1,2015-11-30,02:00:00,2,334,0,Monday,2015-11-30T02:00:00.000000000Z,1062,1.05798,1.05918,1.05770,1.05853
2,2015-11-30,06:00:00,6,334,0,Monday,2015-11-30T06:00:00.000000000Z,4372,1.05856,1.05951,1.05636,1.05810
3,2015-11-30,10:00:00,10,334,0,Monday,2015-11-30T10:00:00.000000000Z,4910,1.05813,1.05849,1.05646,1.05748
4,2015-11-30,14:00:00,14,334,0,Monday,2015-11-30T14:00:00.000000000Z,6511,1.05752,1.05854,1.05578,1.05598


In [16]:
data.tail(5)

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close
9654,2022-02-14,10:00:00,10,45,0,Monday,2022-02-14T10:00:00.000000000Z,49427,1.13122,1.13426,1.13008,1.13152
9655,2022-02-14,14:00:00,14,45,0,Monday,2022-02-14T14:00:00.000000000Z,52445,1.13150,1.13212,1.12916,1.13106
9656,2022-02-14,18:00:00,18,45,0,Monday,2022-02-14T18:00:00.000000000Z,34469,1.13104,1.13144,1.12801,1.13064
9657,2022-02-14,22:00:00,22,45,0,Monday,2022-02-14T22:00:00.000000000Z,7688,1.13064,1.13146,1.13011,1.13106
9658,2022-02-15,02:00:00,2,46,1,Tuesday,2022-02-15T02:00:00.000000000Z,8662,1.13106,1.13226,1.13091,1.13199


## Simple Moving Average (SMA)

In [17]:
data['SMA_5'] = data['Close'].rolling(window=5).mean().round(4)
data['SMA_10'] = data['Close'].rolling(window=10).mean().round(4)
data['SMA_20'] = data['Close'].rolling(window=20).mean().round(4)

## Simple Moving Average Range

In [18]:
data['F_SMA_5'] = data['Close'] - data['SMA_5']
data['F_SMA_10'] = data['Close'] - data['SMA_10']
data['F_SMA_20'] = data['Close'] - data['SMA_20']

In [19]:
data = data.drop_duplicates()
data = data.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [20]:
data.tail()

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close,SMA_5,SMA_10,SMA_20,F_SMA_5,F_SMA_10,F_SMA_20
9654,2022-02-14,10:00:00,10,45,0,Monday,2022-02-14T10:00:00.000000000Z,49427,1.13122,1.13426,1.13008,1.13152,1.1336,1.1364,1.1398,-0.00208,-0.00488,-0.00828
9655,2022-02-14,14:00:00,14,45,0,Monday,2022-02-14T14:00:00.000000000Z,52445,1.13150,1.13212,1.12916,1.13106,1.1329,1.1355,1.1392,-0.00184,-0.00444,-0.00814
9656,2022-02-14,18:00:00,18,45,0,Monday,2022-02-14T18:00:00.000000000Z,34469,1.13104,1.13144,1.12801,1.13064,1.1319,1.1347,1.1386,-0.00126,-0.00406,-0.00796
9657,2022-02-14,22:00:00,22,45,0,Monday,2022-02-14T22:00:00.000000000Z,7688,1.13064,1.13146,1.13011,1.13106,1.1311,1.1339,1.1379,-0.00004,-0.00284,-0.00684
9658,2022-02-15,02:00:00,2,46,1,Tuesday,2022-02-15T02:00:00.000000000Z,8662,1.13106,1.13226,1.13091,1.13199,1.1313,1.1332,1.1374,0.00069,-0.00121,-0.00541


## Price Range

In [21]:
data['O-H'] = data['Open'] - data['High']
data['O-L'] = data['Open'] - data['Low']
data['O-C'] = data['Open'] - data['Close']
data['H-L'] = data['High'] - data['Low']
data['H-C'] = data['High'] - data['Close']
data['L-C'] = data['Low'] - data['Close']

data['Direction'] = data['O-C'].apply(lambda x: 1 if x<0 else 0)

data['col_1'] = data['Open'] - data['Close']

for value in data['col_1']:   
    if value > 0:
        data['col_2'] = data['High'] - data['Open']
        data['col_3'] = data['Close'] - data['Low']
    else:
        data['col_2'] = data['High'] - data['Close']
        data['col_3'] = data['Open'] - data['Low']

#Two Previous Candlesticks 
data['col_4'] = data['col_1'].shift(1)
data['col_5'] = data['col_1'].shift(2)

In [22]:
data = data.dropna()
data = data.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [23]:
data.tail()

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,...,O-C,H-L,H-C,L-C,Direction,col_1,col_2,col_3,col_4,col_5
9635,2022-02-14,10:00:00,10,45,0,Monday,2022-02-14T10:00:00.000000000Z,49427,1.13122,1.13426,...,-0.00030,0.00418,0.00274,-0.00144,1,-0.00030,0.00274,0.00114,0.00394,0.00020
9636,2022-02-14,14:00:00,14,45,0,Monday,2022-02-14T14:00:00.000000000Z,52445,1.13150,1.13212,...,0.00044,0.00296,0.00106,-0.00190,0,0.00044,0.00106,0.00234,-0.00030,0.00394
9637,2022-02-14,18:00:00,18,45,0,Monday,2022-02-14T18:00:00.000000000Z,34469,1.13104,1.13144,...,0.00040,0.00343,0.00080,-0.00263,0,0.00040,0.00080,0.00303,0.00044,-0.00030
9638,2022-02-14,22:00:00,22,45,0,Monday,2022-02-14T22:00:00.000000000Z,7688,1.13064,1.13146,...,-0.00042,0.00135,0.00040,-0.00095,1,-0.00042,0.00040,0.00053,0.00040,0.00044
9639,2022-02-15,02:00:00,2,46,1,Tuesday,2022-02-15T02:00:00.000000000Z,8662,1.13106,1.13226,...,-0.00093,0.00135,0.00027,-0.00108,1,-0.00093,0.00027,0.00015,-0.00042,0.00040


In [24]:
data.shape

(9640, 30)